# Updated Workfow

Following from the draft workflow, I will now define several functions and generic scripts that will help segregate the input/output and the actual functions.

In [1]:
from pybedtools import BedTool # Note this requires bedtools
from glob import glob
import os
import pandas as pd
import numpy as np

In [2]:
# Ensure at top-level directory
os.chdir("../")
os.getcwd()

'/home/work2017/Documents/Jamin'

Current status of dir:

```
.
├── data
│   ├── preprocessed
│   │   ├── markers_lifted
│   │   │   ├── GSM628531_cd4_h2ax_lifted.bed
│   │   │   ├── GSM628531_cd4_h2ax_lifted.err
│   │   │   ├── GSM628531_cd4_h2ax_lifted_sorted.bed
│   │   │   ├── GSM628532_jurkat_h2ax_lifted.bed
│   │   │   ├── GSM628532_jurkat_h2ax_lifted.err
│   │   │   ├── GSM628532_jurkat_h2ax_lifted_sorted.bed
│   │   │   ├── GSM628533_hl60_h2ax_lifted.bed
│   │   │   ├── GSM628533_hl60_h2ax_lifted.err
│   │   │   ├── GSM628533_hl60_h2ax_lifted_sorted.bed
│   │   │   ├── GSM628534_cd4_rgh2ax_lifted.bed
│   │   │   ├── GSM628534_cd4_rgh2ax_lifted.err
│   │   │   ├── GSM628534_cd4_rgh2ax_lifted_sorted.bed
│   │   │   ├── GSM628535_jurkat_gh2ax_lifted.bed
│   │   │   ├── GSM628535_jurkat_gh2ax_lifted.err
│   │   │   └── GSM628535_jurkat_gh2ax_lifted_sorted.bed
│   │   └── mutations_bed
│   │       ├── TCGA.BRCA.muse.6fcfff20-4993-4789-b27b-69d165130466.DR-6.0.somatic.bed
│   │       ├── TCGA.BRCA.muse.6fcfff20-4993-4789-b27b-69d165130466.DR-6.0.somatic_sorted.bed
│   │       ├── TCGA.BRCA.mutect.96983226-d92a-449d-8890-e1b210cee0fe.DR-6.0.somatic.bed
│   │       ├── TCGA.BRCA.mutect.96983226-d92a-449d-8890-e1b210cee0fe.DR-6.0.somatic_sorted.bed
│   │       ├── TCGA.BRCA.somaticsniper.8855730e-21ad-4711-9be2-b58bc337834e.DR-6.0.somatic.bed
│   │       ├── TCGA.BRCA.somaticsniper.8855730e-21ad-4711-9be2-b58bc337834e.DR-6.0.somatic_sorted.bed
│   │       ├── TCGA.BRCA.varscan.053e3955-5fbd-46d3-8a21-c3cde48ef9b4.DR-6.0.somatic.bed
│   │       └── TCGA.BRCA.varscan.053e3955-5fbd-46d3-8a21-c3cde48ef9b4.DR-6.0.somatic_sorted.bed
│   ├── processed
│   └── raw
│       ├── biospecimen_data
│       ├── blacklists
│       │   └── hg38.blacklist.bed
│       ├── clinical_data
│       ├── liftOverchains
│       │   ├── hg18ToHg38.over.chain
│       │   └── hg19ToHg38.over.chain
│       ├── markers
│       │   ├── GSE25577
│       │   │   ├── GSM628531_cd4_h2ax_1.bed
│       │   │   ├── GSM628531_cd4_h2ax_2.bed
│       │   │   ├── GSM628532_jurkat_h2ax_1.bed
│       │   │   ├── GSM628532_jurkat_h2ax_2.bed
│       │   │   ├── GSM628533_hl60_h2ax_1.bed
│       │   │   ├── GSM628533_hl60_h2ax_2.bed
│       │   │   ├── GSM628534_cd4_rgh2ax_1.bed
│       │   │   ├── GSM628534_cd4_rgh2ax_2.bed
│       │   │   └── GSM628535_jurkat_gh2ax.bed
│       │   └── GSE25577_concatenated
│       │       ├── GSM628531_cd4_h2ax.bed
│       │       ├── GSM628532_jurkat_h2ax.bed
│       │       ├── GSM628533_hl60_h2ax.bed
│       │       └── GSM628534_cd4_rgh2ax.bed
│       ├── markers_other
│       │   ├── GSM628530_cd4_h2a_1.bed
│       │   ├── GSM628530_cd4_h2a_2.bed
│       │   └── GSM628537_input_DNA.bed
│       └── mutations
│           ├── TCGA.BRCA.muse.6fcfff20-4993-4789-b27b-69d165130466.DR-6.0.somatic.maf
│           ├── TCGA.BRCA.mutect.96983226-d92a-449d-8890-e1b210cee0fe.DR-6.0.somatic.maf
│           ├── TCGA.BRCA.somaticsniper.8855730e-21ad-4711-9be2-b58bc337834e.DR-6.0.somatic.maf
│           └── TCGA.BRCA.varscan.053e3955-5fbd-46d3-8a21-c3cde48ef9b4.DR-6.0.somatic.maf
└── src
    ├── 20170625_exploratory.ipynb
    ├── 20170627_draftworkflow.ipynb
    ├── 20170628_updatedworkflow.ipynb
    └── 20170629_secondupdatedworkflow.ipynb

16 directories, 50 files

```

# Preprocessing in Bash

## Concatenating .bed files where necessary

Some of the raw files seem to be split into two. Just a quick script to concatenate the two files...

In [31]:
%%bash

cd data/raw/markers/GSE25577

destination=../GSE25577_concatenated/

for filename in *1.bed;
do
    file_no_dir=${filename##*/}
    file_new_name=${file_no_dir/_1.bed/.bed} 
    cp $filename ../GSE25577_concatenated/$file_new_name
    cat ${filename/1.bed/2.bed} >> ../GSE25577_concatenated/$file_new_name
done

## liftOver

In [32]:
%%bash


# LIFTOVER FOR hg18 BED FILES -----------------------------------------------------------------------------
cd data/raw/markers/GSE25577_concatenated
pwd

dir_chain=../../liftOverchains/hg18ToHg38.over.chain
dir_save=../../../preprocessed/markers_lifted/

for filename in *.bed;
do
    liftOver $filename $dir_chain $dir_save${filename/.bed/_lifted.bed} $dir_save${filename/.bed/_lifted.err}
done

/home/work2017/Documents/Jamin/data/raw/markers/GSE25577_concatenated


Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates


> NOTE: ensure to keep hg18 files and hg19 files separated...

## Subsetting .maf Based on Clinical/Biospecimen Features

> Still to be done

## Converting .maf to .bed

In [10]:
%%bash

# Ensure data/ lies in your current directory
cd data/raw/mutations/

for filename in *.maf;
do 
    cut -f 5-8 $filename \
    | awk 'NR >= 6 {print}' \
    | awk '$3 = $3 FS "." FS "."' \
    | sed s/" "/"\t"/g \
    > ../../preprocessed/mutations_bed/${filename%.maf}.bed
done

## Sorting Preprocessed .bed files

Since this is needed for the Fisher test, and appears to be more straightforward in bash, I'll do it here...

In [4]:
%%bash

cd data/preprocessed/markers_lifted/

for filename in *.bed;
do
    sort -k1,1 -k2,2n $filename >> ${filename/.bed/_sorted.bed}
done

In [5]:
%%bash

cd data/preprocessed/mutations_bed/

for filename in *.bed;
do
    sort -k1,1 -k2,2n $filename >> ${filename/.bed/_sorted.bed}
done

# Populating the Blacklist for Filtering

> Still to be done...

# Intersection Function

> Note that this incorporates the filter step within it as well.

In [14]:
def bed_intersect(marker_bed, mutations_bed):
    # INPUTS:
    # marker_bed: BedTool object of the coordinates of the marker
    # mutations_bed: BedTool object of the coordinates of the mutations
    # OUTPUTS:
    # A tuple consisting of:
    # - fisher_test: object representing the output from bedtools fisher test (print to see)
    # CANCELLED - prop_intersection: % of marker_bed that intersected with mutations_bed
    # CANCELLED - intersect_bed: BedTool object of the intersections
    
    # Function set variables ------------------------------------------------------------------
    dir_blacklist = "data/raw/blacklists/"
    name_blacklist = "hg38.blacklist.bed"
    
    window_size = 1000 # size of windows around the points for intersection, if using window
    
    # Filtering -------------------------------------------------------------------------------
    
    # Produce blacklist BedTool instance
    blacklist = BedTool("".join([dir_blacklist, name_blacklist]))
    
    # Filter by subtracting blacklist from the original beds
    filtered_marker_bed = marker_bed.subtract(blacklist)
    filtered_mutations_bed = mutations_bed.subtract(blacklist)
    
    # Intersection ----------------------------------------------------------------------------
    
    # Intersection WITH WINDOW
    # intersect_bed = filtered_marker.window(filtered_mutations_bed, w=window_size)
    
    # Intersection WITHOUT WINDOW
    # Using sorted=True as these should have been presorted...docs say uses faster algorithm...
    intersect_bed = filtered_marker_bed.intersect(filtered_mutations_bed, sorted=True)
    
    # Statistics-------------------------------------------------------------------------------
    
    # Proportion of intersection
    num_intersection = intersect_bed.count()
    num_total = filtered_marker_bed.count()
    prop_intersection = num_intersection / num_total
    
    # For Fisher's exact test
    # Sorting first as bedtools documentations mentions that this is required
    # NOTE: seems that this is sorted such that chr12 comes before chr 2 - is this a problem? 
    # It seems that even the bedtools original would have done this too...
    # NEVERMIND: for the moment, have sorted externally to this script
    # sorted_marker_bed = filtered_marker_bed.sort()
    # sorted_mutations_bed = filtered_mutations_bed.sort()
    fisher_test = filtered_marker_bed.fisher(filtered_mutations_bed, genome="hg38")
    
    # Output ----------------------------------------------------------------------------------
    
    return (fisher_test, prop_intersection, num_intersection, num_total)

There are some questions regarding the intersection that I have:

1. There is the possibility of using the window, but it appears this greatly increases the number of intersections. I wonder if the region of $\gamma$-H2AX is sufficient to include that of the mutations - is the window there in case of random shifts?
2. Does strandedness matter? I'm inclined to think not since it doesn't make sense to me how phosphorylated H2AX could be strand-selective, but there are both positive and negative strands in the bed file so I'm not entirely sure what that means...But then from visual inspection of the mutation data, they all seem positive...
3. There are some statistical tests already implemented in BedTools that might be useful: Fisher exact test seems most relevant, but the Jaccard statistics may be useful too. http://bedtools.readthedocs.io/en/latest/content/tools/fisher.html. I feel that unless there is some reason not to, this would not be a bad idea at the moment to use (at least until a better can be found)
4. See the sorting comment in the code above.

# Python Scripts

At this stage, we have done our preprocessing (producing a new set of BED files that can be used directly from Python with `pybedtools`, and need to stitch things together with the function created above.

The data we have preprocessed should be stored in the directories `data/preprocessed/marks_lifted` and `data/preprocessed/mutations_bed`. 

In [15]:
# INPUT/OUTPUT

dir_markers = "data/preprocessed/markers_lifted/"
dir_mutations = "data/preprocessed/mutations_bed/"

col_names = ["marker_file", "mutation_file", "fisher_stat", "prop_intersection", "num_intersection", "num_total"]

agg_results = pd.DataFrame(columns=col_names)

# As always, need to be wary of the nested for loop...
for file_marker in glob("".join([dir_markers, "*sorted.bed"])):
    for file_mutations in glob("".join([dir_mutations, "*sorted.bed"])):
        print(file_mutations)
        bed_marker = BedTool(file_marker)
        bed_mutations = BedTool(file_mutations)
        results = bed_intersect(bed_marker, bed_mutations)
        
        filename_marker = file_marker.replace(dir_markers, "")
        filename_mutations = file_mutations.replace(dir_mutations, "")
        
        data_entry = [[filename_marker, filename_mutations, results[0], results[1], results[2], results[3]]]
        print(data_entry)
        new_row = pd.DataFrame(data_entry, columns=col_names)
        agg_results = agg_results.append(new_row)

data/preprocessed/mutations_bed/TCGA.BRCA.somaticsniper.8855730e-21ad-4711-9be2-b58bc337834e.DR-6.0.somatic_sorted.bed
[['GSM628533_hl60_h2ax_lifted_sorted.bed', 'TCGA.BRCA.somaticsniper.8855730e-21ad-4711-9be2-b58bc337834e.DR-6.0.somatic_sorted.bed', <FisherOutput at 140077309121144>
# Number of query intervals: 39292264
# Number of db intervals: 65666
# Number of overlaps: 272107
# Number of possible intervals (estimated): 15722781
# phyper(272107 - 1, 39292264, 15722781 - 39292264, 65666, lower.tail=F)
# Contingency Table Of Counts
#_________________________________________
#           |  in -b       | not in -b    |
#     in -a | 272107       | 39020157     |
# not in -a | 0            | 0            |
#_________________________________________
# p-values for fisher's exact test
left	right	two-tail	ratio
1	1	1	-nan
, 0.006925205429750752, 272107, 39292264]]
data/preprocessed/mutations_bed/TCGA.BRCA.mutect.96983226-d92a-449d-8890-e1b210cee0fe.DR-6.0.somatic_sorted.bed
[['GSM628533_h

[['GSM628534_cd4_rgh2ax_lifted_sorted.bed', 'TCGA.BRCA.varscan.053e3955-5fbd-46d3-8a21-c3cde48ef9b4.DR-6.0.somatic_sorted.bed', <FisherOutput at 140077309824976>
# Number of query intervals: 34121180
# Number of db intervals: 88299
# Number of overlaps: 203040
# Number of possible intervals (estimated): 15723461
# phyper(203040 - 1, 34121180, 15723461 - 34121180, 88299, lower.tail=F)
# Contingency Table Of Counts
#_________________________________________
#           |  in -b       | not in -b    |
#     in -a | 203040       | 33918140     |
# not in -a | 0            | 0            |
#_________________________________________
# p-values for fisher's exact test
left	right	two-tail	ratio
1	1	1	-nan
, 0.005950556223436587, 203040, 34121180]]
data/preprocessed/mutations_bed/TCGA.BRCA.muse.6fcfff20-4993-4789-b27b-69d165130466.DR-6.0.somatic_sorted.bed
[['GSM628534_cd4_rgh2ax_lifted_sorted.bed', 'TCGA.BRCA.muse.6fcfff20-4993-4789-b27b-69d165130466.DR-6.0.somatic_sorted.bed', <FisherOutput a

> I quickly had to repeat this for the leftover one that came in an originally complete file...

In [17]:
# As always, need to be wary of the nested for loop...
for file_marker in glob("".join([dir_markers, "*_jurkat_gh2ax_lifted_sorted.bed"])):
    for file_mutations in glob("".join([dir_mutations, "*sorted.bed"])):
        print(file_mutations)
        bed_marker = BedTool(file_marker)
        bed_mutations = BedTool(file_mutations)
        results = bed_intersect(bed_marker, bed_mutations)
        
        filename_marker = file_marker.replace(dir_markers, "")
        filename_mutations = file_mutations.replace(dir_mutations, "")
        
        data_entry = [[filename_marker, filename_mutations, results[0], results[1], results[2], results[3]]]
        print(data_entry)
        new_row = pd.DataFrame(data_entry, columns=col_names)
        agg_results = agg_results.append(new_row)

data/preprocessed/mutations_bed/TCGA.BRCA.somaticsniper.8855730e-21ad-4711-9be2-b58bc337834e.DR-6.0.somatic_sorted.bed
[['GSM628535_jurkat_gh2ax_lifted_sorted.bed', 'TCGA.BRCA.somaticsniper.8855730e-21ad-4711-9be2-b58bc337834e.DR-6.0.somatic_sorted.bed', <FisherOutput at 140077309376496>
# Number of query intervals: 11665241
# Number of db intervals: 65666
# Number of overlaps: 111468
# Number of possible intervals (estimated): 15726766
# phyper(111468 - 1, 11665241, 15726766 - 11665241, 65666, lower.tail=F)
# Contingency Table Of Counts
#_________________________________________
#           |  in -b       | not in -b    |
#     in -a | 111468       | 11553773     |
# not in -a | 0            | 4061525      |
#_________________________________________
# p-values for fisher's exact test
left	right	two-tail	ratio
1	0	0	inf
, 0.009555567690371764, 111468, 11665241]]
data/preprocessed/mutations_bed/TCGA.BRCA.mutect.96983226-d92a-449d-8890-e1b210cee0fe.DR-6.0.somatic_sorted.bed
[['GSM628535

In [18]:
print(agg_results)

                                marker_file  \
0     GSM628533_hl60_h2ax_lifted_sorted.bed   
0     GSM628533_hl60_h2ax_lifted_sorted.bed   
0     GSM628533_hl60_h2ax_lifted_sorted.bed   
0     GSM628533_hl60_h2ax_lifted_sorted.bed   
0   GSM628532_jurkat_h2ax_lifted_sorted.bed   
0   GSM628532_jurkat_h2ax_lifted_sorted.bed   
0   GSM628532_jurkat_h2ax_lifted_sorted.bed   
0   GSM628532_jurkat_h2ax_lifted_sorted.bed   
0    GSM628534_cd4_rgh2ax_lifted_sorted.bed   
0    GSM628534_cd4_rgh2ax_lifted_sorted.bed   
0    GSM628534_cd4_rgh2ax_lifted_sorted.bed   
0    GSM628534_cd4_rgh2ax_lifted_sorted.bed   
0      GSM628531_cd4_h2ax_lifted_sorted.bed   
0      GSM628531_cd4_h2ax_lifted_sorted.bed   
0      GSM628531_cd4_h2ax_lifted_sorted.bed   
0      GSM628531_cd4_h2ax_lifted_sorted.bed   
0  GSM628535_jurkat_gh2ax_lifted_sorted.bed   
0  GSM628535_jurkat_gh2ax_lifted_sorted.bed   
0  GSM628535_jurkat_gh2ax_lifted_sorted.bed   
0  GSM628535_jurkat_gh2ax_lifted_sorted.bed   

            